# Import modules

In [ ]:
import stlab
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import scipy
from scipy.optimize import *
import time
import sys 
import io
import os
from IPython.display import display, Javascript
from shutil import *
from stlab.devices.RS_SGS100A import RS_SGS100A
from stlab.devices.Keysight_N5183B import Keysight_N5183B

from qm.QuantumMachinesManager import QuantumMachinesManager
from qm.qua import *
from qm import SimulationConfig
from Configuration_BMDevice import config, RR_1_IF,RO_lo, readout_len, Q1_lo, Q1_IF, sat_Q1_len, gauss_len, R1_RS, Q1_RS, Q1_pi_len
print(readout_len)
print(Q1_pi_len)

# Edit configuration parameters

**To do**

# Define microwave sources

In [ ]:
RR = RS_SGS100A("TCPIP::169.254.184.193::INSTR", reset=True,verb=True) 
RR.EXTref()
RR.RFon()
RR.setCWpower(-15)
RR.setCWfrequency(RO_lo)
RR.write(':SOURce:IQ:IMPairment:LEAKage:I ' + R1_RS[0])
RR.write('SOURce:IQ:IMPairment:LEAKage:Q ' + R1_RS[1])
RR.write(':SOURce:IQ:IMPairment:IQRatio:MAGNitude ' + R1_RS[2])
RR.write(':SOURce:IQ:IMPairment:QUADrature:ANGLe ' + R1_RS[3])
RR.IQon()
RR.write(':SOURce:IQ:IMPairment:STATe ON')

#LO for downconversion
MXG = Keysight_N5183B(addr='TCPIP::192.168.1.91::INSTR',reset=True,verb=True)
MXG.RFon()
MXG.setCWpower(19)
MXG.setCWfrequency(RO_lo)
MXG.INTref()

QDrive = RS_SGS100A("TCPIP::169.254.50.124::INSTR", reset=True,verb=True) 
QDrive.EXTref()
QDrive.RFon()
QDrive.setCWpower(-35)
QDrive.setCWfrequency(Q1_lo)
QDrive.write(':SOURce:IQ:IMPairment:LEAKage:I ' + Q1_RS[0])
QDrive.write('SOURce:IQ:IMPairment:LEAKage:Q ' + Q1_RS[1])
QDrive.write(':SOURce:IQ:IMPairment:IQRatio:MAGNitude ' + Q1_RS[2])
QDrive.write(':SOURce:IQ:IMPairment:QUADrature:ANGLe ' + Q1_RS[3])
QDrive.IQon()
QDrive.write(':SOURce:IQ:IMPairment:STATe ON')

# Configure QM unit

In [ ]:
qmm = QuantumMachinesManager()
qm = qmm.open_qm(config)

# QUA Program

In [ ]:
def T1(f, n_max, meas_max, tau_vec, a_vec, b, QLO_Power):
    QDrive.setCWpower(QLO_Power)
    
    tau_min = np.min(tau_vec)
    tau_max = np.max(tau_vec)
    dtau = tau_vec[1]-tau_vec[0]
    
    with program() as T1:
        ##############################
        # declare real-time variables:
        ##############################

        n = declare(int)
        meas_n = declare(int)
        tau = declare(int)
        a = declare(fixed)

        A = declare(fixed)
        B = declare(fixed)
        iA = declare(fixed)
        iB = declare(fixed)
        Re = declare(fixed)
        Im = declare(fixed)

        Re_st = declare_stream()
        Im_st = declare_stream()
        
        update_frequency("Q1", f)
        with for_(tau, tau_min, tau <= tau_max, tau+dtau): 
            with for_(meas_n, 0, meas_n < meas_max, meas_n+1):
                with for_(n, 0, n < n_max, n + 1):
                    with for_each_(a, a_vec):
                        
                        wait(int(5000/4),"RR_1","Q1")    
                        with if_(tau < 4):
                            play("pi"*amp(a),"Q1")  
                            align("Q1","RR_1")
                            measure("readout"*amp(b), "RR_1", None, demod.full("integW_cos", A, "out1"),
                                                                            demod.full("integW_sin",B,"out2"),
                                                                            demod.full("integW_sin", iA, "out1"),
                                                                            demod.full("integW_cos",iB,"out2"))
                        with else_(): 
                            play("pi"*amp(a), "Q1")
                            wait(tau, "Q1", "RR_1")                    
                            align("Q1", "RR_1")     
                            measure("readout"*amp(b), "RR_1", None, demod.full("integW_cos", A, "out1"),
                                                                            demod.full("integW_sin",B,"out2"),
                                                                            demod.full("integW_sin", iA, "out1"),
                                                                            demod.full("integW_cos",iB,"out2"))
                        assign(Re, A + B)       
                        assign(Im, iA - iB)
                        save(Re, Re_st)
                        save(Im, Im_st)  

        with stream_processing():
            Re_st.buffer(len(tau_vec),meas_max,n_max,2).map(FUNCTIONS.average(2)).save("Re")
            Im_st.buffer(len(tau_vec),meas_max,n_max,2).map(FUNCTIONS.average(2)).save("Im")

    job = qm.execute(T1, duration_limit=0, data_limit=0)

    res_handles= job.result_handles
    res_handles.wait_for_all_values()

    Re_handle = res_handles.get("Re")
    Im_handle = res_handles.get("Im")

    Re_s = Re_handle.fetch_all()
    Im_s = Im_handle.fetch_all()

    return Re_s, Im_s

# Define Parameters

In [ ]:
#Averaging
n_max = 4000

#Number of measurements at each tau
meas_max = 400 

#Wait time in units of clock cycles (4ns)
tau_min = 2 
tau_max = 2000 
dtau = 100 
tau_vec = np.arange(tau_min,tau_max+dtau/2,dtau)
 
print(tau_vec)
print(len(tau_vec))

#Qubit pulse amplitude
a_vec = [0,1.0]

#Readout pulse amplitude
b = 0.24

#Qubit pulse frequency
f_q = 5.86544e9
qd_IF = int(f_q - Q1_lo)
print(qd_IF/1e6)

#Qubit Pulse Power
QLO_Power = 0

In [ ]:
Re, Im = T1(qd_IF, n_max, meas_max, tau_vec, a_vec, b, QLO_Power)

# Fetch data and save to .dat

In [ ]:
prefix = 'S' #prefix for measurement folder name.  Can be anything or empty
idstring = f'Q1_T1'

meas = np.arange(meas_max)
a = np.repeat(np.array(a_vec)[:,np.newaxis],meas_max,axis=1)
a_arr = np.swapaxes(np.repeat(a[:,:,np.newaxis],len(tau_vec),axis=2),0,2)
tau_arr = ((tau_vec.reshape(len(tau_vec),1)+meas)-meas)
tau_arrf = np.repeat(tau_arr[:,:,np.newaxis],np.shape(a_vec)[0],axis=2)

data_Re = Re
data_Im = Im

data = np.asarray([tau_arrf,a_arr, data_Re,data_Im])

for i,amps in enumerate(meas):
    print(i,amps)
    for j,wt in enumerate(tau_vec):
        print(j,wt)
        data_dict={'Wait time (ns)':data[0][j][i]*4,
                   'Qubit Pulse Amplitude': data[1][j][i],
                   'Real':data[2][j][i],
                   'Imaginary':data[3][j][i],
                  }

        if (i==0 and j==0):
            old_stdout = sys.stdout
            new_stdout = io.StringIO()
            sys.stdout = new_stdout

            myfile=stlab.newfile(prefix,idstring,data_dict.keys(),autoindex=True, git_id = False)

            output = new_stdout.getvalue()
            sys.stdout = old_stdout
            print(output)
            M_ind = output.find("Measurement Name")
            M_name = output[M_ind+len('Measurement Name:  '):-1]
        stlab.savedict(myfile,data_dict)
        stlab.metagen.fromarrays(myfile,tau_vec[0:i+1]*4,(np.array(a_vec))[0:i+1],[0,0],xtitle='Wait time (ns)',ytitle='Qubit Pulse Amplitude',colnames=list(data_dict))

# Processing/Plotting

In [ ]:
def s_std(dI,std_dI,dQ,std_dQ):
    return np.sqrt((dI**2*std_dI**2+dQ**2*std_dQ**2)/(dI**2+dQ**2))

def Contrast(Re,Im):
    sg = [Re[:,:,0],Im[:,:,0]]
    se = [Re[:,:,1],Im[:,:,1]]

    sg_mean = np.mean(sg,axis=2)
    sg_std = np.std(sg,axis=2)/np.sqrt(meas_max)

    se_mean = np.mean(se,axis=2)
    se_std = np.std(se,axis=2)/np.sqrt(meas_max)

    dI = se_mean[0]-sg_mean[0]
    dQ = se_mean[1]-sg_mean[1]

    std_dI = np.sqrt(se_std[0]**2+sg_std[0]**2)
    std_dQ = np.sqrt(se_std[1]**2+sg_std[1]**2)

    s_a = np.sqrt(dI**2+dQ**2)
    s_a_std = s_std(dI,std_dI,dQ,std_dQ)

    C = s_a/s_a[0]
    std_C = C*np.sqrt((s_a_std[:]/s_a[:])**2+(s_a_std[0]/s_a[0])**2)
    
    return C, std_C

C, std_C = Contrast(Re,Im)

In [ ]:
#plt.plot(tau_vec*4,C)
tau_vec[0] = 0 

def P1_func(tau,T1):
    return np.exp(-tau/T1)           

popt, pcov = curve_fit(P1_func, tau_vec*4, C,sigma=std_C, p0 = [800])

plt.figure(figsize = (8,6), dpi = 100)

tau_fit = np.linspace(0,8000,1600)
plt.title('T1, Q_t = {} ns, Q_amp={}, t_RO={}ns, R_amp = {}, Qfreq  = {} GHz, QLO_Power = {} dBm'.format(Q1_pi_len, a_vec[1],readout_len,b, f_q/1e9,QLO_Power)) 
plt.plot(tau_fit,P1_func(tau_fit,popt[0]), label = f'fit, T1 = {np.round(popt[0],1)} $\pm$ {np.round(np.sqrt(np.diag(pcov))[0],0)} ns')
plt.errorbar(tau_vec*4,C, std_C,color = 'orange',capsize = 5,marker = 'o',ls='',alpha = 0.75)
plt.xlabel(r' $\tau$ (ns)', fontsize = 16)
plt.ylabel(r'$C$', fontsize = 16)
plt.xticks(fontsize = 14); plt.yticks(fontsize = 14);
plt.legend(fontsize = 14)
plt.axhline(0,ls='--', color ='black')

print(f'T1 = {popt[0]} ns')
print(np.sqrt(np.diag(pcov))[0])

In [ ]:
plt.figure(figsize=(8,6),dpi=100)
plt.scatter(Re[0,:,0],Im[0,:,0], s = 1)
plt.scatter(Re[0,:,1],Im[0,:,1], s = 1)

plt.scatter(Re[4,:,0],Im[4,:,0], s = 1)
plt.scatter(Re[4,:,1],Im[4,:,1], s = 1)

plt.scatter(Re[17,:,0],Im[17,:,0], s = 1)
plt.scatter(Re[17,:,1],Im[17,:,1], s = 1)
plt.axhline(0, c='black', ls = '--')
plt.axvline(0, c='black', ls = '--')

# Save this file and configuration file to measurement folder

In [ ]:
#save notebook
display(Javascript('IPython.notebook.save_checkpoint();'))

In [ ]:
%%javascript
IPython.notebook.kernel.execute('nb_name = "' + IPython.notebook.notebook_name + '"')

In [ ]:
#define document paths
meas_path = os.path.join(os.getcwd(),M_name)

current_nb_path = os.path.join(os.getcwd(),nb_name)
save_nb_path = os.path.join(meas_path,nb_name)

current_config_path = os.path.join(os.getcwd(), 'Configuration_BMDevice.py')
save_config_path = os.path.join(meas_path, 'Configuration_BMDevice.py')

#copy to measurement folder 
copy2(current_nb_path,save_nb_path);
copy2(current_config_path,save_config_path);

#### Simulation

In [ ]:
a1 = 0.5 #1.55
b = 0.24

with program() as sim_prog:
    a = declare(fixed)
    x = declare(int)
    tau = declare(int)
    
    assign(tau,10)
    assign(a, a1)
    
    with if_(tau < 4):
        play("pi"*amp(a),"Q1")  
        align("Q1","RR_1")
        measure("readout"*amp(b), "RR_1", "raw_adc")
    with else_(): 
        play("pi"*amp(a), "Q1")
        wait(tau, "Q1", "RR_1")                    
        align("Q1", "RR_1")     
        measure("readout"*amp(b), "RR_1", "raw_adc")

#    align("Q1", "RR_1")
#    measure("readout"*amp(b), "RR_1", "raw_adc")
        

# In the OPX, the analog signal starts 184 after the play command. In order to simulate it, we added the same latency
# here, and this is the time_of_flight in the configuration file
job = qmm.simulate(config,
    sim_prog,
    SimulationConfig(
        500,
        include_analog_waveforms=True,
    ),latency=184
)

# get DAC and digital samples
samples = job.get_simulated_samples()
print(job.simulated_analog_waveforms())

# plot all ports:
plt.figure(figsize = (8,6),dpi=100)
plt.plot(samples.con1.analog["1"], label = 'Readout Pulse')
plt.plot(samples.con1.analog["3"],alpha = 1, label = 'Qubit Pulse')
plt.legend("analog 1")
plt.xlabel("Time [ns]")
plt.ylabel("Signal [V]")
plt.xlim(200,750)
plt.axvline(272, ls = '--', lw = 1, c ='black')
plt.axvline(368, ls = '--', lw = 1, c ='black')
plt.axvline(408, ls = '--', lw = 1, c ='black')
plt.legend()

# samples = job.get_simulated_samples()
# res = job.result_handles
# raw_adc = res.raw_adc_input1.fetch_all()["value"]

# ax1 = plt.subplot(211)
# plt.plot(samples.con1.analog["1"])
# plt.title("Simulated samples")
# plt.subplot(212, sharex=ax1)
# plt.plot(raw_adc / 2 ** 12)  # Converting the 12 bit ADC value to voltage
# plt.title("Raw ADC input")
# plt.xlabel("Sample number")
# plt.tight_layout()
# plt.show()